In [159]:
import numpy as np
from music21 import converter, instrument, note, chord, meter, duration, stream
import music21

In [ ]:
"""We modeled the monophonic melodies and basslines as
sequences of 16th note events. This resulted in a 130-
dimensional output space (categorical distribution over tokens) with 128 “note-on” tokens for the 128 MIDI pitches,
plus single tokens for “note-off” and “rest”. For drum patterns, we mapped the 61 drum classes defined by the General
MIDI standard (International MIDI Association, 1991) to 9
canonical classes and represented all possible combinations
of hits with 512 categorical tokens. For timing, in all cases
we quantized notes to 16th note intervals, such that each
bar consisted of 16 events. As a result, our two-bar data
(used as a proof-of-concept with a flat decoder) resulted in
sequences with T = 32 and 16-bar data had T = 256. For
our hierarchical models, we use U = 16, meaning each
subsequence corresponded to a single bar"""

In [47]:
ts = meter.TimeSignature('3/4')
print(ts.numerator)
print(ts.denominator)

3
4


In [204]:
#midi_path = 'data/midis/8th_allnotes_oneinstrument.mid'
#midi_path = 'data/midis/Rod_Bass_-_Quantized.mid'
midi_path = 'data/acdc/Sin_City.mid'


midi = converter.parse(midi_path)
ts = midi.getTimeSignatures()[0]
num = ts.numerator
denom = ts.denominator

measure_length = None # length of measure in 16ths
single_note_length = None # length of a single note in 16ths

if denom / 4 == 1:
    measure_length = num * 4 # quarter notes
    single_note_length = 4
elif denom / 4 == 2:
    measure_length = num * 2 # eigth notes
    single_note_length = 2
elif denom / 4 == 4:
    measure_length = num # 16ths
    single_note_length = 1


notes_to_parse = None
parts = instrument.partitionByInstrument(midi)
if parts: # file has instrument parts
    notes_to_parse = parts.parts[1].recurse()
    parts.parts[1].makeMeasures(inPlace=True)
    measureMap = parts.parts[1].measureOffsetMap()
else: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

#current_instrument = measureMap[0.0][0].instrument
#data = encode_midi([measureMap[0.0], measureMap[4.0]], t=128)

""" for element in notes_to_parse:
    break
    if isinstance(element, note.Note):
        print("{}\t{}\t{}\t{}\t{}".format(element, element.pitch.midi, element.beat, element.duration, element.measureNumber))
    elif isinstance(element, chord.Chord):
        #element.duration.type = 'eighth'
        print("{}\t{}".format(element, [p.midi for p in element.pitches]))
    else:
        print(element) """

' for element in notes_to_parse:\n    break\n    if isinstance(element, note.Note):\n        print("{}\t{}\t{}\t{}\t{}".format(element, element.pitch.midi, element.beat, element.duration, element.measureNumber))\n    elif isinstance(element, chord.Chord):\n        #element.duration.type = \'eighth\'\n        print("{}\t{}".format(element, [p.midi for p in element.pitches]))\n    else:\n        print(element) '

In [208]:
def insert_note(arr, note, offset_16th): 
    idx = int(offset_16th + (note.beat - 1) * single_note_length)
    arr[int(note.pitch.midi), idx] = True

    note_off = idx + int(note.duration.quarterLength * 4) - 1
    if note_off >= arr.shape[1]:
        note_off = arr.shape[1] - 1
    arr[-1, note_off] = True

def encode_midi(measureMap, t=32, header=None):
    data = np.zeros((130, t), dtype=np.bool)
    measure_count = 0
    fill_header = True

    for i in range(0, t, measure_length):
        for element in measureMap[i / 4.0][0]:
            #print(element)
            if isinstance(element, note.Note):
                fill_header = False
                insert_note(data, element, i)
            elif fill_header and header != None:
                print(type(element))
                header.append(element)

            """elif isinstance(element, music21.clef.TrebleClef) or isinstance(element, music21.clef.BassClef):
                clf = element
            elif isinstance(element, music21.tempo.MetronomeMark):
                tempo = element
            elif isinstance(element, music21.key.Key):
                song_key = element
            elif isinstance(element, music21.instrument.Instrument):
                inst = element
            elif isinstance(element, music21.meter.TimeSignature):
                ts = element
            else:
                print(type(element))"""
    
    return data

def decode_midi(data, header=None):
    strm = stream.Stream()
    
    for element in header[1:]: # TODO: right now I'm skipping the instrument because it adds like 10 empty measures for some reason
        strm.append(element)

    current_note = None
    current_length = 0.25

    for t in range(data.shape[1]):
        nothing = True

        for p, val in enumerate(data[:,t]):
            if val:
                if p < 128:
                    current_note = note.Note()
                    current_note.pitch.midi = p
                    current_length = 0.25

                    nothing = False
                elif p == 128:
                    pass #TODO: something with rests
                else:
                    if nothing:
                        current_length += 0.25
                    current_note.duration = duration.Duration(current_length)
                    #print('{}\t{}'.format(current_note.pitch, current_note.duration))
                    strm.append(current_note)
                    current_note = None
                    nothing = False
        
        if nothing and current_note: # if there was no data in this time step and there is a current note increase it's length by 1 16th
            current_length += 0.25
    
    return strm


In [209]:
header = []
measures = 16
data = encode_midi(measureMap, t=measures*16, header=header)
strm = decode_midi(data, header)
strm.ticksPerQuarterNote = 1024
strm.makeRests(fillGaps=True, inPlace=True)
#strm.show('text')
strm.write('midi', fp='data/Sin_City_decoded.mid')

<class 'music21.instrument.ElectricBass'>
<class 'music21.clef.Bass8vbClef'>
<class 'music21.tempo.MetronomeMark'>
<class 'music21.key.Key'>
<class 'music21.meter.TimeSignature'>


'data/Sin_City_decoded.mid'

In [138]:
n = note.Note('D#')
n.beat = 1.00
print(n.pitch.midi)

AttributeError: can't set attribute